In [12]:
import numpy as np
import pandas as pd

In [74]:
input_fname = '../Train_Tagged_Titles/Train_Tagged_Titles.tsv'

In [98]:
df = pd.read_csv(input_fname, sep='\t')

In [99]:
df.drop(['Unnamed: 4'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [100]:
# df.head(30)
print(df.iloc[0])

Record Number                                                    1
Title            Supreme Nike SB Dunk High By any Means Red US1...
Token                                                      Supreme
Tag                                                         Modell
Name: 0, dtype: object


In [101]:
print(len(df))

52005


In [102]:
lim = len(df)
i = 0

while i < lim:    
    row = df.iloc[i]
    
    if row['Tag'] is np.NaN:
        df.iloc[i-1]['Token'] = str(df.iloc[i-1]['Token']) + ' ' + str(row['Token'])
        df.drop([i], inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        lim = len(df)
    else:
        i += 1